<img src="img/LeetCode.png" style="width: 300px;"/>

***
# SQL Exercises - LeetCode
<br>

1. [Disclaimer](#disclaimer)
1. [Relevant Information](#info)
1. [Imports](#imports)
1. [Connections](#connection)
1. [Exercises](#Exercises)
    - [Easy Level](#easy)
    - [Medium Level](#medium)
    - [Hard Level](#hard)

<a id=disclaimer></a>

## Disclaimer
***

<div class="span5 alert alert-danger">
    <b>Note:</b> These exercises come from LeetCode they are from the free tier. Their service is great so I recommend anyone seeing this notebook to instead head over to their <a href=https://leetcode.com>website</a> and try them there. This notebook has been helpful for me to practice SQL.
</div>

[Completely Uninstall & Install PostgreSQL](https://medium.com/@bitadj/completely-uninstall-and-reinstall-psql-on-osx-551390904b86)

<a id=info></a>

## Relevant Information
***


**I will be using PostgreSQL to practice**

Here are some of the basic commands for macOS users

- `brew install postgresql` --> will install postgresql
- `initdb /usr/local/var/postgres` --> will point to the data directory
- `psql -U postgres` --> will ask for the password to enter your database
- `\du` --> will show the users
- `\l` --> will show the existing db
- `CREATE DATABASE leetcode;` --> will create the database with the name leetcode (see complete syntax below)
- `\c leetcode` --> will enter the database
- `\q` --> will close the connection to Postgres
- `CREATE TABLE tb_name;` --> Will create a table in your database
- `DROP TABLE tb_name;` --> Will delete a table from your database

**Complete syntax to create database**<br>
`CREATE DATABASE db_name
OWNER =  role_name
TEMPLATE = template
ENCODING = encoding
LC_COLLATE = collate
LC_CTYPE = ctype
TABLESPACE = tablespace_name
CONNECTION LIMIT = max_concurrent_connection`

<a id=imports></a>

## Imports
***

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import sqlalchemy


In [2]:
from sqlalchemy import Table, Column, Integer, String, MetaData, VARCHAR, insert, update
from sqlalchemy.orm import sessionmaker

<a id=connection></a>

## Connection
***

In [3]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_ADDRESS = 'localhost' 
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' 
POSTGRES_PASSWORD = 'LCmd2020!'
POSTGRES_DBNAME = 'leetcode' 

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
                                                                                        password=POSTGRES_PASSWORD,
                                                                                        ipaddress=POSTGRES_ADDRESS,
                                                                                        port=POSTGRES_PORT,
                                                                                        dbname=POSTGRES_DBNAME))
# Create the connection
engine = create_engine(postgres_str) 
Session = sessionmaker(bind=engine)
session = Session()

<a id=Exercises></a>

## Exercises

<a id=easy></a>

### Level Easy
***

#### 197. Rising Temperature

**Information:** Write an SQL query to find all dates' id with higher temperature compared to its previous dates (yesterday).

`CREATE TABLE weather(
id Serial PRIMARY KEY, 
recorddate date,
temperature int);`


`INSERT INTO weather(recorddate,temperature)
VALUES ('2015-01-01'::date, 10),
('2015-01-02'::date, 25),
('2015-01-03'::date, 20),
('2015-01-04'::date, 30);`

In [4]:
pd.read_sql_query('''SELECT * FROM weather;''', engine)

,id,recorddate,temperature
0,1,2015-01-01,10
1,2,2015-01-02,25
2,3,2015-01-03,20
3,4,2015-01-04,30


In [5]:
pd.read_sql_query('''
SELECT t1.*, 
        LAG(t1.temperature) OVER (ORDER by recorddate) AS is_bigger_than_previous
FROM weather t1
;''', 
                  
engine)

,id,recorddate,temperature,is_bigger_than_previous
0,1,2015-01-01,10,NaN
1,2,2015-01-02,25,10.0
2,3,2015-01-03,20,25.0
3,4,2015-01-04,30,20.0


In [6]:
pd.read_sql_query('''
SELECT t.id, t.recorddate, t.temperature
FROM (
  SELECT t1.*, 
         t1.temperature > LAG(t1.temperature) OVER (ORDER by recorddate) AS is_bigger_than_previous
  FROM weather t1
) t
WHERE is_bigger_than_previous
;''', 
                  
engine)

,id,recorddate,temperature
0,2,2015-01-02,25
1,4,2015-01-04,30


In [7]:
pd.read_sql_query('''
SELECT *, DATE_PART('day', weather.recorddate) - DATE_PART('day', w.recorddate)
FROM
    weather
        JOIN
    weather w ON weather.id = w.id
;''', 
                  
engine)

,id,recorddate,temperature,id,recorddate,temperature,?column?
0,1,2015-01-01,10,1,2015-01-01,10,0.0
1,2,2015-01-02,25,2,2015-01-02,25,0.0
2,3,2015-01-03,20,3,2015-01-03,20,0.0
3,4,2015-01-04,30,4,2015-01-04,30,0.0


In [8]:
pd.read_sql_query('''
SELECT w2.id AS "id"
FROM weather w1
JOIN weather w2
    ON EXTRACT(DAY FROM CAST(w2.recorddate AS TIMESTAMP)-CAST(w1.recorddate AS TIMESTAMP)) = 1
        AND w2.temperature > w1.temperature;
;''', 
                  
engine)

,id
0,2
1,4


#### 596. Classes More Than 5 Student

**Information:** There is a table courses with columns: student and class. Please list out all classes which have more than or equal to 5 students.

`CREATE TABLE courses(
student VARCHAR(2),
classes VARCHAR(20);`

`INSERT INTO courses(student,classes
VALUES ('A', 'Math'),
('B', 'English'),
('C', 'Math'),
('D', 'Biology'),
('E', 'Math'),
('F', 'Computer'),
('G', 'Math'),
('H', 'Math'),
('I', 'Math');`

In [92]:
pd.read_sql_query('''
SELECT classes 
FROM courses 
GROUP BY classes 
HAVING COUNT(DISTINCT student) >= 5;
;''', 
                  
engine)

,classes
0,Math


#### 627. Swap Salary

Given a table `salary`, such as the one below, that has m=male and f=female values. Swap all f and m values (i.e., change all f values to m and vice versa) with a single update statement and no intermediate temp table.

`CREATE TABLE salary(
id serial PRIMARY KEY,
name VARCHAR(2),
sex VARCHAR(1),
salary INT);`

`INSERT INTO salary(name,sex,salary)
VALUES ('A', 'm',2500),
('B', 'f',1500),
('C', 'm',5500),
('D', 'f',500)`

In [87]:
pd.read_sql_query('''
SELECT *, CASE WHEN sex = 'f' THEN 'm'
                WHEN sex = 'm' THEN 'f'
               END AS change 
FROM salary;''', 
                  
engine)

,id,name,sex,salary,change
0,1,A,m,2500,f
1,2,B,f,1500,m
2,3,C,m,5500,f
3,4,D,f,500,m


In [ ]:
pd.read_sql_query('''
UPDATE salary
SET sex = CASE 
                 WHEN sex = 'f' THEN 'm'
                 WHEN sex = 'm' THEN 'f'
               end;
''',                   
engine)

In [89]:
pd.read_sql_query('''
SELECT *, CASE WHEN sex = 'f' THEN 'm'
                WHEN sex = 'm' THEN 'f'
               END AS change 
FROM salary;''', 
                  
engine)

,id,name,sex,salary,change
0,1,A,f,2500,m
1,2,B,m,1500,f
2,3,C,f,5500,m
3,4,D,m,500,f


#### 1179. Reformat Department Table

**Instructions:** Write an SQL query to reformat the table such that there is a department id column and a revenue column for each month.

I will use `crosstab`from the `tablefunc`module: `CREATE EXTENSION IF NOT EXISTS tablefunc;`

`CREATE TABLE department(
id int NOT NULL,
revenue int,
month VARCHAR NOT NULL,
PRIMARY KEY(id, month));`

`INSERT INTO department(id, revenue, month)
VALUES 
(1, 8000, 'Jan'),
(2, 9000,'Jan'),
(3, 10000,'Feb'),
(1, 7000,'Feb'),
(1, 6000,'Mar');`

In [247]:
df = pd.read_sql_query('''
SELECT *
FROM department;''', 
                  
engine)

In [251]:
df

,id,revenue,month
0,1,8000,Jan
1,2,9000,Jan
2,3,10000,Feb
3,1,7000,Feb
4,1,6000,Mar


In [256]:
pd.crosstab(index= df.id, columns=df.month, values=df.revenue, aggfunc=sum)

month,Feb,Jan,Mar
id,,,
1,7000.0,8000.0,6000.0
2,NaN,9000.0,NaN
3,10000.0,NaN,NaN


In [234]:
pd.read_sql_query('''
SELECT *
FROM crosstab(
   'SELECT id, month, revenue
    FROM   department
    ORDER  BY 1,2'  
   ) AS ct ("id" int, "Jan" int, "Feb" int, "Mar" int);
''', 
                  
engine)

,id,Jan,Feb,Mar
0,1,7000,8000.0,6000.0
1,2,9000,NaN,NaN
2,3,10000,NaN,NaN


In [285]:
pd.read_sql_query('''
CREATE OR REPLACE VIEW history_extended as ( -- Can be done with a cte statement as well
  select
    *,
    case when month = 'Jan' then revenue end as A,
    case when month = 'Feb' then revenue end as B,
    case when month = 'Mar' then revenue end as C,
    case when month = 'Apr' then revenue end as D,
    case when month = 'May' then revenue end as E,
    case when month = 'Jun' then revenue end as F,
    case when month = 'Jul' then revenue end as G,
    case when month = 'Aug' then revenue end as H,
    case when month = 'Sep' then revenue end as I,
    case when month = 'Oct' then revenue end as J,
    case when month = 'Nov' then revenue end as K,
    case when month = 'Dec' then revenue end as L
  from department
);

SELECT id, SUM(A) as Jan_Revenue, SUM(B) as Feb_Revenue, SUM(C) as Mar_Revenue,
        SUM(D) as Apr_Revenue, SUM(E) as May_Revenue, SUM(F) as Jun_Revenue,
        SUM(G) as Jul_Revenue, SUM(H) as Aug_Revenue, SUM(I) as Sep_Revenue,
        SUM(J) as Oct_Revenue, SUM(K) as Nov_Revenue, SUM(L) as Dec_Revenue
FROM history_extended
GROUP BY id
ORDER BY id ASC;
;''', 
                  
engine)

,id,jan_revenue,feb_revenue,mar_revenue,apr_revenue,may_revenue,jun_revenue,jul_revenue,aug_revenue,sep_revenue,oct_revenue,nov_revenue,dec_revenue
0,1,8000.0,7000.0,6000.0,None,None,None,None,None,None,None,None,None
1,2,9000.0,NaN,NaN,None,None,None,None,None,None,None,None,None
2,3,NaN,10000.0,NaN,None,None,None,None,None,None,None,None,None


<a id=easy></a>

### Medium Level
***

#### 626. Exchange Seats

**Instructions:** Mary is a teacher in a middle school and she has a table seat storing students' names and their corresponding seat ids. The column id is continuous increment.

`CREATE TABLE seat(
id serial PRIMARY KEY,
name VARCHAR(20));`

`INSERT INTO seat(name)
VALUES ('Abbot'),
    ('Doris'),
    ('Emerson'),
    ('Green'),
    ('Jeames');`

In [95]:
pd.read_sql_query('''
SELECT * 
FROM seat;''', 
                  
engine)

,id,name
0,1,Abbot
1,2,Doris
2,3,Emerson
3,4,Green
4,5,Jeames


**NOT CORRECT**

In [217]:
pd.read_sql_query('''
WITH new as (SELECT id, 
                    CASE WHEN id=(SELECT COUNT(*) FROM SEAT) THEN id 
                        WHEN mod(id,2)=0 THEN id-1 
                        WHEN mod(id,2)=1 THEN id+1 
                        END as new_id
FROM seat
ORDER BY id ASC)

SELECT new_id as id, name
FROM seat
INNER JOIN new ON seat.id = new.id
ORDER BY new_id ASC
;''', 
                  
engine)

,id,name
0,1,Doris
1,2,Abbot
2,3,Green
3,4,Emerson
4,5,Jeames


<a id=easy></a>

### Hard Level
***

### Notes
***

In [ ]:
# Syntax to delete wrongly added information
#pd.read_sql_query('''DELETE FROM table
#WHERE id IN (5,6,7,8)
#RETURNING *;''', cnx)

In [69]:
#engine = create_engine(postgres_str)
#metadata = MetaData(bind=engine)

#courses = Table(
 #  'courses', metadata, 
  # Column('student', VARCHAR(2)), #primary_key = True), 
   #Column('classes', String), 
#)

#courses.drop(engine, checkfirst=True)
#metadata.create_all(engine)

In [ ]:
#i = insert(courses)
#i = i.values({'student':"A", 'classes':'Math'})
#session.execute(i)

In [ ]:
#courses.insert().values([{'student':"A", 'classes':'Math'},
 #                       {'student':"B", 'classes':'English'}])